In [ ]:
import pandas as pd
import numpy as np

import os
import ntpath
import re
import time


from stat import *
from rdflib import *
from rdflib.namespace import *

import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
prefix = URIRef("http://www.semanticweb.org/d19125691/ontologies/2020/7/CSVDQ")
owl = URIRef("http://www.w3.org/2002/07/owl#")
rdf = URIRef("https://www.w3.org/1999/02/22-rdf-syntax-ns")
dc = URIRef("http://purl.org/dc/elements/1.1/")

In [ ]:
#Data to triples
g = Graph()
dataset = prefix+"#dataset"
fileSize = prefix+"/fileSize"
fileMod = prefix+"/fileModified"
fformat = dc+"/format"

data = pd.read_csv("/home/d19125691/Experiments/Experiments/datasets/example.csv") 

#adding triples to graph g
for i in range(len(data)):
    rowprefix=prefix+"/R"+str(i)
    g.add((rowprefix, RDF.type, prefix))
    for k in range(len(data.columns)):
        value=data.iloc[i][k]
        colprefix = prefix+":"+data.columns[k]
        g.add((rowprefix, colprefix, Literal(value)))

#adding metadata to graph g     
try:
    st = os.stat("/home/d19125691/Experiments/Experiments/datasets/example.csv")
except IOError:
    print("failed to get information about")
else: 
    g.add((dataset, fileSize, Literal(st[ST_SIZE])))
    g.add((dataset, fileMod, Literal(time.asctime(time.localtime(st[ST_MTIME])))))
    g.add((dataset, fformat, Literal("text/csv") ))

g.serialize("data.rdf", format="n3")

In [ ]:
#Dynamic ontology
g1 = Graph()

classpre = owl+"Class"
thing = owl+"Thing"

about = rdf +"about"
rowpre = prefix+"#row"
colpre = prefix+"#columns"
dataset = prefix+"#dataset"

#dataset info
#g1.add((dataset, RDFS.subClassOf, thing))
#g1.add((dataset, RDF.type, classpre))

#row information
g1.add((rowpre, RDF.type, classpre))
g1.add((rowpre, RDFS.subClassOf, dataset))

for i in range(len(data)):
    rowprefix=prefix+"/R"+str(i)
    g1.add((rowprefix,  RDF.type , rowpre))
    
#column information
g1.add((colpre, RDF.type, classpre))
g1.add((colpre, RDFS.subClassOf, dataset))


print(g1.serialize(format='n3').decode("utf-8"))
print(g.serialize(format='n3').decode("utf-8"))

g1.serialize("ontology.rdf", format="n3")